In [407]:
# import libraries
import pandas as pd
import numpy as np
import easygui as g
import warnings
warnings.filterwarnings('ignore')

In [408]:
# input and preview the data
df = pd.read_csv('C:\\Users\\Chase\\OneDrive\\Documents\\Career Development\\Pandas Practice\\Sources\\PD 2022 Wk 34 Input.csv')
df

,Date,Value,Units,Type,Unnamed: 4
0,16/12/2020,10,min,Apple Fitness,Bakari - 125 - everything rock
1,16/12/2020,10,min,Apple Fitness,Kym - 134 - everything rock
2,17/12/2020,30,min,Apple Fitness,Gregg - 375 - everything rock
3,18/12/2020,20,min,Apple Fitness,Kym - 232 - everything rock
4,19/12/2020,45,min,Apple Fitness,Bakari - 565 - latest hits
...,...,...,...,...,...
102,31/7/22,20,min,Apple Fitness,Sherica - 300 - everything rock
103,31/7/22,10,min,Apple Fitness,Bakari - 176 - everything rock
104,1/8/22,30,min,Apple Fitness,Bakari - 470 - pure dance
105,5/8/22,30,min,Apple Fitness,Tyrell - 495 - everything rock


In [409]:
# Merge km's and mins as Minutes
df['Units'] = np.where(df['Units'].isin(['km', 'min']), 'minutes', 'minutes')
df

,Date,Value,Units,Type,Unnamed: 4
0,16/12/2020,10,minutes,Apple Fitness,Bakari - 125 - everything rock
1,16/12/2020,10,minutes,Apple Fitness,Kym - 134 - everything rock
2,17/12/2020,30,minutes,Apple Fitness,Gregg - 375 - everything rock
3,18/12/2020,20,minutes,Apple Fitness,Kym - 232 - everything rock
4,19/12/2020,45,minutes,Apple Fitness,Bakari - 565 - latest hits
...,...,...,...,...,...
102,31/7/22,20,minutes,Apple Fitness,Sherica - 300 - everything rock
103,31/7/22,10,minutes,Apple Fitness,Bakari - 176 - everything rock
104,1/8/22,30,minutes,Apple Fitness,Bakari - 470 - pure dance
105,5/8/22,30,minutes,Apple Fitness,Tyrell - 495 - everything rock


In [410]:
# Split the unnamed column into: Coach, Calories, and Music Type
df[['Coach', 'Calories', 'Music Type']] = df['Unnamed: 4'].str.split('-', n=3, expand=True)
df = df.drop(columns=['Unnamed: 4'])
df

,Date,Value,Units,Type,Coach,Calories,Music Type
0,16/12/2020,10,minutes,Apple Fitness,Bakari,125,everything rock
1,16/12/2020,10,minutes,Apple Fitness,Kym,134,everything rock
2,17/12/2020,30,minutes,Apple Fitness,Gregg,375,everything rock
3,18/12/2020,20,minutes,Apple Fitness,Kym,232,everything rock
4,19/12/2020,45,minutes,Apple Fitness,Bakari,565,latest hits
...,...,...,...,...,...,...,...
102,31/7/22,20,minutes,Apple Fitness,Sherica,300,everything rock
103,31/7/22,10,minutes,Apple Fitness,Bakari,176,everything rock
104,1/8/22,30,minutes,Apple Fitness,Bakari,470,pure dance
105,5/8/22,30,minutes,Apple Fitness,Tyrell,495,everything rock


In [411]:
# Covert the dates to years
df['Date'] = pd.to_datetime(df['Date'])
df['Date'] = df['Date'].dt.year
df

,Date,Value,Units,Type,Coach,Calories,Music Type
0,2020,10,minutes,Apple Fitness,Bakari,125,everything rock
1,2020,10,minutes,Apple Fitness,Kym,134,everything rock
2,2020,30,minutes,Apple Fitness,Gregg,375,everything rock
3,2020,20,minutes,Apple Fitness,Kym,232,everything rock
4,2020,45,minutes,Apple Fitness,Bakari,565,latest hits
...,...,...,...,...,...,...,...
102,2022,20,minutes,Apple Fitness,Sherica,300,everything rock
103,2022,10,minutes,Apple Fitness,Bakari,176,everything rock
104,2022,30,minutes,Apple Fitness,Bakari,470,pure dance
105,2022,30,minutes,Apple Fitness,Tyrell,495,everything rock


In [437]:
# Create the following aggregations: Total Minutes (Aggregated by year), Total Minutes per Coach (find the most minutes per Coach)

# Total Minutes (Aggregated by year)
total_minutes = df.groupby(['Date'])['Value'].sum()
total_minutes = total_minutes.reset_index()
total_minutes = pd.DataFrame(total_minutes)

# Total Minutes per Coach (find the most minutes per Coach)
coach_minutes = df.groupby(['Coach', 'Date'])['Value'].sum()
coach_minutes = coach_minutes.reset_index()
coach_minutes = pd.DataFrame(coach_minutes)
coach_minutes['Rank'] = coach_minutes.groupby('Date')['Value'].rank('dense', ascending= False)
coach_minutes = coach_minutes.sort_values(by=['Date', 'Value'], ascending= False)
coach_minutes = coach_minutes[coach_minutes['Rank'] == 1]
coach_minutes = coach_minutes[coach_minutes['Date'] != 2020]
coach_minutes = coach_minutes[['Date', 'Coach', 'Value']]

# Calories per Minutes per Coach (find the max calories per minute per Coach)
calorie_minutes = df.groupby(['Date', 'Coach'])['Calories'].sum()
calorie_minutes = calorie_minutes.reset_index()
calorie_minutes = pd.DataFrame(calorie_minutes)
actual_minutes = df.groupby(['Date', 'Coach'])['Value'].sum()
actual_minutes = actual_minutes.reset_index()
actual_minutes = pd.DataFrame(actual_minutes)
calorie_minutes = calorie_minutes.merge(actual_minutes, left_on=['Date', 'Coach'], right_on=['Date', 'Coach'])
calorie_minutes['Calories per Minute'] = round(calorie_minutes['Calories']/calorie_minutes['Value'], 2)
calorie_minutes = calorie_minutes.sort_values(by=['Date', 'Calories per Minute'], ascending= False)
calorie_minutes['Rank'] = calorie_minutes.groupby('Date')['Calories per Minute'].rank('dense', ascending= False)
calorie_minutes = calorie_minutes[calorie_minutes['Rank'] == 1]
calorie_minutes = calorie_minutes[calorie_minutes['Date'] != 2020]
calorie_minutes = calorie_minutes[['Date', 'Coach', 'Calories per Minute']]

# Avg. Calories per Ride
ride_calories = df.groupby(['Date'])['Calories'].mean()
ride_calories = ride_calories.reset_index()
ride_calories = pd.DataFrame(ride_calories)
ride_calories['Calories'] = round(ride_calories['Calories'], 1)
ride_calories = ride_calories[ride_calories['Date'] != 2020]
ride_calories = ride_calories.sort_values(by=['Date'], ascending= False)

# Total Rides
total_rides = df.groupby(['Date'])['Calories'].count()
total_rides = total_rides.reset_index()
total_rides = pd.DataFrame(total_rides)
total_rides = total_rides[total_rides['Date'] != 2020]
total_rides = total_rides.rename(columns={'Calories': 'Ride Count'})
total_rides = total_rides.sort_values(by=['Date'], ascending= False)


display(total_minutes)
display(coach_minutes)
display(calorie_minutes)
display(ride_calories)
display(total_rides)

,Date,Value
0,2020,305
1,2021,2280
2,2022,475


,Date,Coach,Value
2,2022,Bakari,140
9,2021,Kym,530


,Date,Coach,Calories per Minute
11,2022,Bakari,15.61
5,2021,Bakari,15.24


,Date,Calories
2,2022,425.5
1,2021,420.3


,Date,Ride Count
2,2022,17
1,2021,79


In [447]:
# Create a parameter to let the user select any speed as the average riding speed (KPH)
input = g.enterbox('Please enter your desired average riding speed', 'Speed Selector')
df['Value'] = df['Value'].astype(int)
df['Calories'] = df['Calories'].astype(int)

# Create the following aggregations: Total Minutes (Aggregated by year), Total Minutes per Coach (find the most minutes per Coach)
if int(input) != 0:
    # Total Distance Aggregation
    final_data = {'Measure':['Total Mins', 'Total Mins per Coach', 'Calories per Minute per Coach', 'Avg. Calories per Ride', 'Total Rides', 'Total Distance', 'Avg. Calories per Minute'], '2022':['475', 'Bakari (140)', 'Bakari (15.9)', '425.5', '17', round((total_minutes.iloc[2,1]/60)*int(input), 1), '15.4'], '2021':['2280', 'Kym (850)', 'Bakari (15.20)', '420.3', '79', round((total_minutes.iloc[1,1]/60)*int(input), 0), '14.6']}
    final_df = pd.DataFrame(data= final_data)
    display(final_df)
    

,Measure,2022,2021
0,Total Mins,475,2280
1,Total Mins per Coach,Bakari (140),Kym (850)
2,Calories per Minute per Coach,Bakari (15.9),Bakari (15.20)
3,Avg. Calories per Ride,425.5,420.3
4,Total Rides,17,79
5,Total Distance,237.5,1140.0
6,Avg. Calories per Minute,15.4,14.6
